In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sid import get_simulate_func
from sid.time import get_date

from src.config import BLD
from src.contact_models.get_contact_models import (
    get_all_contact_models,
    get_christmas_contact_models,
)
from src.create_initial_states.create_initial_conditions import (
    create_initial_conditions,
)
from src.policies.combine_policies_over_periods import get_december_to_feb_policies

# Inputs that won't change

In [2]:
params = pd.read_pickle(BLD / "start_params.pkl")
params.to_csv(BLD / "start_params.csv")

share_known_cases = pd.read_pickle(
    BLD / "data" / "processed_time_series" / "share_known_cases.pkl"
)

# Inputs that will change

In [3]:
# replace this with the full initial states later on
df = pd.read_parquet(BLD / "data" / "debug_initial_states.parquet")

# contact_models and policies will change depending on the
# scenario in which we are.
# Parametrize over 3 christmas_modes and
# private_contact_tracing True/False to begin with
contact_models = get_all_contact_models(
    christmas_mode="full", n_extra_contacts_before_christmas=2
)

# policies
policies = get_december_to_feb_policies(
    contact_models=contact_models,
    pre_christmas_other_multiplier=0.3,
    christmas_other_multiplier=0.0,
    post_christmas_multiplier=0.3,
    contact_tracing_multiplier=0.1,
)

# This is only until we have estimated infection probabilities!

In [4]:
def _build_infection_probs(names):
    index_tuples = [("infection_prob", mod_name, mod_name) for mod_name in names]
    df = pd.DataFrame(index=pd.MultiIndex.from_tuples(index_tuples))
    df.index.names = ["category", "subcategory", "name"]
    df = df.reset_index()
    prob_dict = {
        "educ": 0.02,
        "work": 0.1,
        "household": 0.2,
        "other": 0.1,
        "christmas": 0.2,
        "holiday_preparation": 0.1,
    }
    full_prob_dict = {}
    for mod_name in names:
        for k, v in prob_dict.items():
            if k in mod_name:
                full_prob_dict[mod_name] = v
        assert (
            mod_name in full_prob_dict
        ), f"No infection probability for {mod_name} specified."

    df["value"] = df["name"].map(full_prob_dict.get)
    df = df.set_index(["category", "subcategory", "name"])
    return df

In [5]:
# set the infection probabilities
infection_probs = _build_infection_probs(contact_models.keys())

params = pd.concat([infection_probs, params])
params.head()

value
category       subcategory      name                   
infection_prob households       households         0.20
               educ_school_0    educ_school_0      0.02
               educ_school_1    educ_school_1      0.02
               educ_school_2    educ_school_2      0.02
               educ_preschool_0 educ_preschool_0   0.02

# Create the initial conditions

In [6]:
simulation_start = pd.Timestamp("2020-12-01")
initial_start = simulation_start - pd.Timedelta(weeks=2)
simulation_end = simulation_start + pd.Timedelta(weeks=4)

initial_conditions = create_initial_conditions(
    start=initial_start,
    end=simulation_start - pd.Timedelta(days=1),
    reporting_delay=7,
    seed=99,
)

# Run the simulation

In [7]:
simulate = get_simulate_func(
    params=params,
    initial_states=df,
    contact_models=contact_models,
    duration={"start": simulation_start, "end": simulation_end},
    contact_policies=policies,
    initial_conditions=initial_conditions,
    events=None,
    seed=384,
)

In [8]:
res = simulate(params)

Start the simulation...
2020-12-29: 100%|██████████| 29/29 [00:26<00:00,  1.08it/s]


In [11]:
df = res["time_series"]
df

,needs_icu,dead,hh_size,work_weekly_group_id_7,occupation,other_weekly_group_id_0,private_hh,preschool_group_id_0,systemically_relevant,school_group_a,christmas_group_id_1,work_weekly_group_id_8,age,infectious,work_weekly_group_id_3,symptomatic,immune,newly_deceased,cd_infectious_false,n_has_infected,hh_model_group_id,date,school_group_id_0,work_daily_group_id,work_weekly_group_id_4,work_weekly_group_id_6,other_weekly_group_id_2,newly_infected,work_weekly_group_id_2,work_weekly_group_id_13,gender,age_group_rki,educ_worker,school_group_id_1,work_weekly_group_id_9,work_weekly_group_id_10,ever_infected,other_daily_group_id,work_weekly_group_id_12,work_contact_priority,other_weekly_group_id_3,nursery_group_id_0,county,new_known_case,other_weekly_group_id_1,work_weekly_group_id_1,knows_infectious,knows_immune,work_weekly_group_id_0,state,christmas_group_id_2,work_weekly_group_id_11,christmas_group_id_0,hh_id,age_group,school_group_id_2,work_weekly_group_id_5
npartitions=29,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,bool,bool,category[unknown],category[unknown],category[unknown],category[unknown],bool,category[unknown],bool,uint8,category[unknown],category[unknown],uint8,bool,category[unknown],bool,bool,bool,int16,uint16,category[unknown],datetime64[ns],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],bool,category[unknown],category[unknown],object,category[unknown],bool,category[unknown],category[unknown],category[unknown],bool,category[unknown],category[unknown],float64,category[unknown],category[unknown],category[unknown],bool,category[unknown],category[unknown],bool,bool,category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# Visualize Results